Instalações e importações necessárias

In [ ]:
!pip install gTTS
!pip install SpeechRecognition pydub
!pip install pydub
!pip install googlesearch-python

In [ ]:
from gtts import gTTS
import os
import speech_recognition as sr
from pydub import AudioSegment
from googlesearch import search

## Transformação de texto em áudio (text to speech);



A função text_to_speech_gtts utiliza a biblioteca gTTS (Google Text-to-Speech) para converter um texto em áudio no idioma português e salvar o resultado como um arquivo MP3. O texto a ser convertido é recebido como parâmetro, e o arquivo gerado é salvo com o nome especificado (padrão: "output.mp3"). Essa função é simples e eficiente para criar arquivos de áudio a partir de textos fornecidos.

In [8]:
def text_to_speech_gtts(text, filename="output.mp3"):
    """
    Converte texto em áudio usando gTTS e salva como arquivo MP3.

    Args:
        text (str): O texto a ser convertido em áudio.
        filename (str): O nome do arquivo de saída. Padrão é "output.mp3".
    """
    # Configurar a conversão do texto em áudio
    tts = gTTS(text=text, lang="pt", slow=False)

    # Salvar o arquivo de áudio
    tts.save(filename)
    print(f"Áudio salvo como {filename}")


Áudio salvo como exemplo.mp3


In [10]:
text = "Ellie e Joel, comidinha."
text_to_speech_gtts(text, "comidinha.mp3")

Áudio salvo como comidinha.mp3


## Transformação de fala (linguagem natural humana) em texto (speech to text)

Aqui foram utilizadas duas funções para trabalhar com áudio. A função speech_to_text utiliza a biblioteca SpeechRecognition para converter um arquivo de áudio nos formatos .wav ou .flac em texto, utilizando a API do Google Speech e suportando o idioma português. Ela captura erros comuns, como áudio não reconhecido ou falhas de conexão. Já a função convert_m4a_to_wav usa a biblioteca pydub para converter arquivos de áudio no formato .m4a para .wav, permitindo que sejam processados por outras funções.

In [15]:
def speech_to_text(audio_file):
    """
    Converte um arquivo de áudio em texto usando SpeechRecognition.

    Args:
        audio_file (str): Caminho para o arquivo de áudio (.wav ou .flac).

    Returns:
        str: Texto transcrito do áudio.
    """
    # Inicializa o reconhecedor de fala
    recognizer = sr.Recognizer()

    try:
        # Carrega o áudio
        with sr.AudioFile(audio_file) as source:
            print("Processando o áudio...")
            audio = recognizer.record(source)  # Lê o áudio

        # Realiza o reconhecimento (usando Google Speech API)
        text = recognizer.recognize_google(audio, language="pt-BR")  # Ajuste o idioma se necessário
        print("Texto transcrito:")
        return text

    except sr.UnknownValueError:
        return "Não foi possível reconhecer o áudio."
    except sr.RequestError as e:
        return f"Erro ao conectar ao serviço: {e}"

def convert_m4a_to_wav(input_file, output_file):
    """Converts an m4a file to wav format.

    Args:
        input_file: Path to the input m4a file.
        output_file: Path to the output wav file.
    """
    try:
        sound = AudioSegment.from_file(input_file, format="m4a")
        sound.export(output_file, format="wav")
        print(f"Successfully converted {input_file} to {output_file}")
    except FileNotFoundError:
        print(f"Error: Input file {input_file} not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [17]:
input_m4a = "Gravando.m4a"
convert_m4a_to_wav(input_m4a, "audio_path.wav")
transcription = speech_to_text(audio_path)
print(transcription)

Successfully converted Gravando.m4a to audio_path.wav
Processando o áudio...
Texto transcrito:
pesquisar a farmácia mais próxima


## Assistente de pesquisa no google

O código realiza uma pesquisa no Google com base em um texto fornecido e retorna os links dos primeiros resultados. A função get_google_results utiliza a biblioteca googlesearch-python para buscar até um número especificado de resultados (padrão: 5) no idioma português. Após executar a busca com o texto o programa exibe os links encontrados em uma lista numerada. Ele também lida com erros gerais, retornando uma mensagem caso algo dê errado durante a busca.

In [21]:
def get_google_results(query, num_results=5):
    """
    Realiza uma pesquisa no Google e retorna os links dos resultados.

    Args:
        query (str): O texto a ser pesquisado.
        num_results (int): Número de resultados desejados. Padrão é 5.

    Returns:
        list: Uma lista com os links dos resultados.
    """
    try:
        # Realiza a pesquisa e coleta os resultados
        results = list(search(query, num_results=num_results, lang="pt"))
        return results
    except Exception as e:
        return [f"Erro ao buscar no Google: {e}"]

text = "como construir um modelo de machine learning"
results = get_google_results(text)

print("Resultados da pesquisa:")
for i, result in enumerate(results, 1):
    print(f"{i}. {result}")


Resultados da pesquisa:
1. https://www.insightlab.ufc.br/6-passos-para-criar-seu-primeiro-projeto-de-machine-learning/
2. https://www.cienciaedados.com/os-7-passos-para-construir-modelos-de-machine-learning/
3. https://learn.microsoft.com/pt-br/training/paths/create-machine-learn-models/
4. https://www.databricks.com/br/glossary/machine-learning-models
5. https://itanuromero.medium.com/di%C3%A1rio-de-desenvolvimento-de-software-cria%C3%A7%C3%A3o-de-um-modelo-de-machine-learning-7dac26153461


## Assistente de pesquisa no google com resultado da transcrição do áudio para texto

A seguir são integrados os processos: conversão de áudio, transcrição de texto e pesquisa no Google. Primeiro, ele converte um arquivo de áudio que foi dado como input no formato .m4a para .wav usando a função convert_m4a_to_wav, necessária para compatibilidade com a transcrição. Em seguida, utiliza a função speech_to_text para transcrever o conteúdo do áudio para texto, empregando a biblioteca SpeechRecognition. Se a transcrição for bem-sucedida, o texto gerado é usado como consulta na função get_google_results, que realiza uma pesquisa no Google e retorna links dos resultados encontrados, exibidos numerados no console. Caso a transcrição falhe, uma mensagem de erro é exibida.

In [26]:
input_m4a = "input.m4a"
convert_m4a_to_wav(input_m4a, "audio_path.wav")
transcription = speech_to_text("audio_path.wav")

if transcription:
    print(transcription)

    # Passo 2: Pesquisa no Google
    results = get_google_results(transcription)

    # Exibe os resultados
    print("\nResultados da pesquisa no Google:")
    for i, result in enumerate(results, 1):
        print(f"{i}. {result}")
else:
    print("Texto não foi transcrito com sucesso.")


Successfully converted input.m4a to audio_path.wav
Processando o áudio...
Texto transcrito:
Defina um modelo de machine learning e Deep lorn

Resultados da pesquisa no Google:
1. https://www.zendesk.com.br/blog/machine-learning-and-deep-learning/
2. https://www.opencadd.com.br/blog/machine-learning-ou-deep-learning
3. https://posdigital.pucpr.br/blog/machine-learning-deep-learning
4. https://www.databricks.com/br/glossary/machine-learning-models
5. https://aws.amazon.com/pt/compare/the-difference-between-machine-learning-and-deep-learning/
